In [ ]:
import random
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from IPython.display import Image, display

**Data Processing (Sequential)** 

In [ ]:
class State(TypedDict):
    input: int
    output: int

In [ ]:
def node_a(state: State)->State:
    input_value = state['input']
    offset = random.randint(1,10)
    output =  input_value + offset
    print(
        f"NODE A:\n "
        f"->input:{input_value}\n " 
        f"->offset:{offset}\n "
        f"->output:{output}\n "
    )
    return State(output=output)


In [ ]:
def node_b(state: State):
    input_value = state['output']
    offset = random.randint(1,10)
    output =  input_value + offset
    print(
        f"NODE B:\n "
        f"->input:{input_value}\n " 
        f"->offset:{offset}\n "
        f"->output:{output}\n "
    )
    return {"output": output}

In [ ]:
workflow = StateGraph(state_schema=State)

In [ ]:
workflow.add_node(node_a)
workflow.add_node(node_b)

In [ ]:
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

In [ ]:
graph = workflow.compile()

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
graph.invoke(
    input = {
        "input": 1,
    }, 
)

**Call LLMs**

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [ ]:
class State(TypedDict):
    question:str
    response:str

In [ ]:
def model(state: State):
    question = state["question"]
    response = llm.invoke([
        SystemMessage("You're a Pokémon specialist"),
        HumanMessage(question)
    ])

    return {"response": response.content}

In [ ]:
workflow = StateGraph(State)

In [ ]:
workflow.add_node("model", model)

In [ ]:
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

In [ ]:
graph = workflow.compile()

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
result = graph.invoke(
    input={
        "question": "What's the name of Ash's first pokémon?"
    }, 
)

In [ ]:
result